# Polya Urn Experiments

To determine if the Polya urn model is appropriate for our application, we need to compare the evolution of our model's FPR to the evolution of the ratio of positive and negative labels in our dataset.

The main (potential) issue with our current setup of the urn model is that the probability of sampling a given class changes. Though this could be interpreted as representing the fact that it's not just new samples who's labels can switch, but also previous true negatives may become false positives since we are forcing the model to fit biased noisy labels.

Specifically, this is what we are modelling 

$$
    \begin{align}
        D^{(t)} &:= \mathrm{Dataset \: at \: time \: t}\\
        D^{(t)}_{TN} &:= \{\mathrm{x_{i} \in D^{(t)} | f(x_{i}) = 0 \land y_i = 0}\}
    \end{align}
$$

$$
    \begin{equation}
        p(f(x_i) = 1 | y_i = 0) = p(fp) / p(y_i = 0)
    \end{equation}
$$

In [ ]:
import numpy as np

## True Random Process

In [49]:
iterations = 1000000
init_samples = 10000
pos_p = 0.5
neg_p = 1 - pos_p

fpr = 0.01
tpr = 0.49
fnr = 0.01
tnr = 0.49

switch_prob = 0.99

model_preds = np.array([int(init_samples * fpr), int(init_samples * tpr), int(init_samples * fnr), int(init_samples * tnr)])
labels = np.array([int(init_samples * neg_p), int(init_samples * pos_p)])

In [50]:
for i in range(iterations):
    new_sample = np.random.choice([0, 1], 1, p=[neg_p, pos_p])
    
    if new_sample == 0:
        p_1_0 = fpr / neg_p
        pred = np.random.choice([0, 1], 1, p=[1 - p_1_0, p_1_0])
        
        if pred == 1:
            model_preds[0] += 1
            labels[1] += 1
            
            switch = np.random.choice([0, 1], 1, p=[1 - switch_prob, switch_prob])
            
            if switch == 1 and model_preds[3] > 0:
                model_preds[3] -= 1
                model_preds[0] += 1
        else:
            model_preds[3] += 1
            labels[0] += 1
    else:
        p_0_1 = fnr / pos_p
        pred = np.random.choice([0, 1], 1, p=[p_0_1, 1 - p_0_1])
        
        if pred == 1:
            model_preds[1] += 1
        else:
            model_preds[2] += 1

        labels[1] += 1
        
    fpr = model_preds[0] / np.sum(model_preds)
    fnr = model_preds[2] / np.sum(model_preds)
    
    if i % 1000 == 0:
        print("Ratio of positive labels: {} | FPR: {}".format(float(labels[1]) / (labels[0] + labels[1]), fpr))
        
        

Ratio of positive labels: 0.5000499950005 | FPR: 0.009999000099990002
Ratio of positive labels: 0.501136260339969 | FPR: 0.01127170257249341
Ratio of positive labels: 0.502374802099825 | FPR: 0.013082243146404467
Ratio of positive labels: 0.5027305591877548 | FPR: 0.013768171679101607
Ratio of positive labels: 0.5045353903292622 | FPR: 0.015498892936218842
Ratio of positive labels: 0.5034997666822212 | FPR: 0.01566562229184721
Ratio of positive labels: 0.5045934629085682 | FPR: 0.016686457096431472
Ratio of positive labels: 0.5051467560731722 | FPR: 0.016763719781189342
Ratio of positive labels: 0.5055274706960724 | FPR: 0.018387867340703294
Ratio of positive labels: 0.5057102257775906 | FPR: 0.019104257670648914
Ratio of positive labels: 0.5063246837658117 | FPR: 0.01974901254937253
Ratio of positive labels: 0.506785391171849 | FPR: 0.020713299366696823
Ratio of positive labels: 0.5084768874142085 | FPR: 0.020680878141902642
Ratio of positive labels: 0.5091082996391462 | FPR: 0.021781

In [47]:
model_preds[3]

1

In [46]:
np.sum(model_preds)

509939

## Polya Random Process

In [11]:
iterations = 100000
init_samples = 10000
pos_p = 0.5
neg_p = 1 - pos_p

fpr = 0.01
tpr = 0.49
fnr = 0.01
tnr = 0.49

labels = np.array([int(init_samples * neg_p), int(init_samples * pos_p)])

In [12]:
for i in range(iterations):
    pos_p = float(labels[1]) / np.sum(labels)
    neg_p = 1 - pos_p
    
    new_sample = np.random.choice([0, 1], 1, p=[neg_p, pos_p])
    
    if new_sample == 1:
        labels[1] += 1
    else:
        switch = np.random.choice([0, 1], 1, [1 - fpr, fpr])
        
        if switch == 1:
            labels[1] += 1
        else:
            labels[0] += 1
            
    if i % 100 == 0:
        print("Ratio of positive labels: {}".format(float(labels[1]) / (labels[0] + labels[1])))

Ratio of positive labels: 0.5000499950005
Ratio of positive labels: 0.503019503019503
Ratio of positive labels: 0.5057347318890305
Ratio of positive labels: 0.5079118532181341
Ratio of positive labels: 0.5103355446591674
Ratio of positive labels: 0.5126178459194363
Ratio of positive labels: 0.5145740967833223
Ratio of positive labels: 0.5168675824689282
Ratio of positive labels: 0.5198592722896028
Ratio of positive labels: 0.5216035226126043
Ratio of positive labels: 0.5239523679665485
Ratio of positive labels: 0.5252679938744257
Ratio of positive labels: 0.5276314614766539
Ratio of positive labels: 0.5303955402176799
Ratio of positive labels: 0.5327602841855977
Ratio of positive labels: 0.5349100078254065
Ratio of positive labels: 0.5371950693905698
Ratio of positive labels: 0.53935561063157
Ratio of positive labels: 0.5413947970510974
Ratio of positive labels: 0.542811528442988
Ratio of positive labels: 0.5449545871177401
Ratio of positive labels: 0.5465663994711181
Ratio of positive

## Naive Bayes

To properly model the expected degredation of a classifier, we have to assume some parametric form for both the classifier and the data. Let's assume that 

$$
    \begin{equation}
        p(y=0)=p(y=1)=0.5 \\
        p(x | y = 0) = \mathcal{N}(-1, 1/4) \\
        p(x | y = 1) = \mathcal{N}(1, 1/4)
    \end{equation}
$$

We assume concrete value for the true mean and std for now, but will make these arbitrary later. Now we need to determine the right estimated mean, and std to make sure our NB classifier has a desired FPR. The decision boundary happens where $p(y = 0 | x) = p(y = 1 | x)$ which can be simplified to

$$
    \begin{align}
        \frac{1}{\sigma_{0}} e^{-(\frac{x - \mu_{0}}{\sigma_{0}})^2} = \frac{1}{\sigma_{1}} e^{-(\frac{x - \mu_{1}}{\sigma_{1}})^2}
    \end{align}
$$

We assume that $\sigma_{0} = \sigma_{1}$ otherwise the math gets very messy. Using this assumption, solving for $x$ gives $x = \frac{\mu_{0} + \mu_{1}}{2}$. Now if we fix $\overline{x}_1$ and, we know that using inverse Z table the value of the decision boundary to have a 1% FPR is $-0.4175$, then $\overline{x}_{0} = 2 * (-0.4175) - \overline{x}_{1}$. W.L.G. we can assume that $\overline{x}_{1} = \mu_{1}$ to get $\overline{x}_{0} = 2 * (-0.4175) - 1 = -1.8350$

*** Scratch what was stated above for the solution of the decision boundary! The above solution assumes that the ratio of pos/neg samples stays constant, but by definition in our case it won't since we are adding noisy labels. Mind you, we keep the probability of *sampling* pos/neg classes constant, but their respective ratio in our dataset up to some point will not be the same. Here is the full solution

$$
    \begin{equation}
        \mu_{0} = x +/- \sqrt{\mu_{1}^{2} -2\mu_{1}x + x^{2} - \sigma_{0}^2 \mathrm{Log} \frac{a}{b}}
    \end{equation}
$$

though in our case, just the negative solution makes sense.

So what happens when we get a new sample of data and want to update the model

In [123]:
from scipy.stats import norm

In [170]:
y0 = 0.5
y1 = 0.5

s0 = 1.0
s1 = 1.0

m0 = -1.0
m1 = 1.0

n = 100
sample_m0 = - 0.44
sample_m1 = m1

fpr = 0.1
gamma = 0.28

In [171]:
n_update = 1000000
cur_data_size = n
n_neg = y0 * n
n_pos = y1 * n

for i in range(100):
    
    sample_m0 = (n_update * (0.5 - fpr) * (m0) + (cur_data_size * p_neg) * sample_m0) / (n_update * (0.5 - fpr) + (cur_data_size * p_neg))
    sample_m1 = (n_update * (0.5*m1 + fpr * m0) + (cur_data_size * p_pos) * sample_m1) / (n_update * (0.5 + fpr) + (cur_data_size * p_pos))
    
    cur_data_size += n_update
    
    n_pos += n_update * (y1 + fpr)
    n_neg += n_update * (y0 - fpr)

    p_pos = n_pos / (cur_data_size)
    p_neg = n_neg / (cur_data_size)
    
    gamma = (sample_m0 ** 2 - sample_m1 ** 2 + (s0 ** 2) * np.log((p_neg / p_pos))) / (2 * (sample_m0 - sample_m1))
    
    fpr = 1.0 - norm.cdf((gamma - m0) / s0)


In [172]:
sample_m0

-0.9999994399088229

In [173]:
sample_m1

0.507171520879883

In [174]:
gamma

-0.021188617905140057

In [175]:
p_neg

0.3365054735810638

In [176]:
fpr

0.16383659236537662

In [177]:
p_pos

0.6634945264189359

In [117]:
n_pos = y1 * n + n_update * (y1 + fpr)
n_neg = y0 * n + n_update * (y0 - fpr)

p_pos = n_pos / (n + n_update)
p_neg = n_neg / (n + n_neg)

Now compute new decision boundary and it's FPR given the fact that our dataset is no longer balanced.

In [118]:
new_d = (sample_m0 ** 2 - sample_m1 ** 2 + (1.0 ** 2) * np.log((p_neg / p_pos))) / (2 * (sample_m0 - sample_m1))

In [119]:
new_d

0.11420532090110772

## Logistic Regression

Let's do the same thing for LR that we did for our NB analysis. There is no closed form for LR, so we'll have to update using GD. We need to check that the gradient on a large sample of data is close to the expected gradient computed using numerical integration via mathematica

### Gaussian Data

`sklearn.datasets.make_classification` creates data that is not quite normally distributed, so to ensure that the compounding FPR is not an artefact of that data, we first test it out on Gaussian data from distributions we define. We also look at how the number of features affects the rate at which the FPR compounds

In [368]:
import copy
import numpy as np
import torch
from sklearn.metrics import confusion_matrix
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [399]:
def make_gaussian_data(m0, m1, s0, s1, n, p0, p1, features=2, flip=0.0):
    neg_samples = np.random.multivariate_normal(m0 * np.ones(features), s0 * np.eye(features), int(n * p0))
    pos_samples = np.random.multivariate_normal(m1 * np.ones(features), s1 * np.eye(features), int(n * p1))
    
    x = np.concatenate((neg_samples, pos_samples))
    y = np.concatenate((np.zeros(len(neg_samples)), np.ones(len(pos_samples))))
    
    idx = np.random.choice(len(x), len(x), replace=False)
    x = x[idx]
    y = y[idx]
    
    return torch.from_numpy(x).float(), torch.from_numpy(y).float()

In [370]:
def eval_model(y, y_pred):
    c_matrix = confusion_matrix(y, y_pred)
    tn, fp, fn, tp = c_matrix.ravel()

    samples = float(len(y_pred))
    
    return tn / samples, fp / samples, fn / samples, tp / samples

In [371]:
def sigmoid(z):
    return 1.0 / (1 + torch.exp(-z))

In [397]:
def train(x_train, y_train):
    w = torch.from_numpy(np.random.randn(x_train.shape[1])).float()
    w.requires_grad = True
    b = torch.tensor(0.0, requires_grad=True).float()

    criterion = torch.nn.BCELoss()
    lr = 1.0

    for i in range(1000):
        out = sigmoid(torch.matmul(x_train, w) + b)

        loss = criterion(out, y_train)

        if i % 500 == 0:
            print(loss)

        loss.backward()
        w.data = w.data - lr * w.grad
        b.data = b.data - lr * b.grad

        w.grad.zero_()
        b.grad.zero_()
        
    return w, b

In [393]:
def update_model_online(w, b, num_updates):
    new_w = copy.deepcopy(w)
    new_b = copy.deepcopy(b)
    
    print(new_w.requires_grad)
    
    size = float(len(y_test1)) / float(num_updates)
    
    prediction_differences = []

    for i in range(num_updates):
        idx_start = int(size * i)
        idx_end = int(size * (i + 1))
        sub_x = x_test1[idx_start: idx_end, :]
        sub_y = copy.deepcopy(y_test1[idx_start: idx_end])

        sub_pred = sigmoid(torch.matmul(sub_x, new_w) + new_b)
        sub_pred_copy = copy.deepcopy(sub_pred.data)
        sub_pred_copy[sub_pred_copy >= 0.5] = 1
        sub_pred_copy[sub_pred_copy < 0.5] = 0
        
        fp_idx = np.logical_and(sub_y == 0, sub_pred_copy == 1)
        sub_y[fp_idx.bool()] = 1
        
        loss = criterion(sub_pred, sub_y)
        loss.backward()
        new_w.data = new_w.data - lr * new_w.grad
        new_b.data = new_b.data - lr * new_b.grad

        new_w.grad.zero_()
        new_b.grad.zero_()
        
    return new_w, new_b

In [394]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# pos_idx = y_train[:1000] == 1
# pos_idx = np.where(pos_idx)
# neg_idx = y_train[:1000] == 0
# neg_idx = np.where(neg_idx)

# plt.scatter(x_train[pos_idx, 0].detach(), x_train[pos_idx, 1].detach(), label=1)
# plt.scatter(x_train[neg_idx, 0].detach(), x_train[neg_idx, 1].detach(), label=0)
# plt.legend()

In [395]:
m0 = -1
m1 = 1

s0 = 1
s1 = 1

n = 1000

p0 = 0.5
p1 = 1 - p0

n_features = 1

In [403]:
n_features = [1, 2, 3, 4, 5]
np.random.seed(1)

for n_feature in n_features:
    print("----- Gaussian data with {} features -----".format(n_feature))
    x_train, y_train = make_gaussian_data(m0 - 0.2, m1 - 0.1, s0, s1, n, p0, p1, features=n_feature)

    x_test1, y_test1 = make_gaussian_data(m0, m1, s0, s1, 1000000, p0, p1, features=n_feature)
    x_test2, y_test2 = make_gaussian_data(m0, m1, s0, s1, 1000000, p0, p1, features=n_feature)
    
    x_train.requires_grad = True
    x_test1.requires_grad = True
    
    w, b = train(x_train, y_train)

    y_pred = sigmoid(torch.matmul(x_test2, w) + b)
    y_pred[y_pred >= 0.5] = 1
    y_pred[y_pred < 0.5] = 0
    tn, fp, fn, tp = eval_model(y_test2, y_pred.detach())
    
    print("Original TPR: {}".format(tp))
    print("Original FPR: {}".format(fp))
    print("Original TNR: {}".format(tn))
    print("Original FNR: {}".format(fn))
    
    new_w, new_b = update_model_online(w, b, 2000)
    
    y_pred = sigmoid(torch.matmul(x_test2, new_w) + new_b)
    y_pred[y_pred >= 0.5] = 1
    y_pred[y_pred < 0.5] = 0
    tn, fp, fn, tp = eval_model(y_test2, y_pred.detach())
    
    print("Updated TPR: {}".format(tp))
    print("Updated FPR: {}".format(fp))
    print("Updated TNR: {}".format(tn))
    print("Updated FNR: {}".format(fn))

----- Gaussian data with 1 features -----
tensor(1.4887, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.3334, grad_fn=<BinaryCrossEntropyBackward>)
Original TPR: 0.431803
Original FPR: 0.091434
Original TNR: 0.408566
Original FNR: 0.068197
True
Updated TPR: 0.495925
Updated FPR: 0.325954
Updated TNR: 0.174046
Updated FNR: 0.004075
----- Gaussian data with 2 features -----
tensor(0.3750, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.1513, grad_fn=<BinaryCrossEntropyBackward>)
Original TPR: 0.470685
Original FPR: 0.051921
Original TNR: 0.448079
Original FNR: 0.029315
True
Updated TPR: 0.48897
Updated FPR: 0.103469
Updated TNR: 0.396531
Updated FNR: 0.01103
----- Gaussian data with 3 features -----
tensor(2.0499, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0855, grad_fn=<BinaryCrossEntropyBackward>)
Original TPR: 0.486886
Original FPR: 0.031798
Original TNR: 0.468202
Original FNR: 0.013114
True
Updated TPR: 0.491653
Updated FPR: 0.045363
Updated TNR: 0.454637
Updated FNR: 0.008347
---

### `sklearn.make_classification` Data

In [411]:
n_features = [1, 2, 3, 4, 5]
np.random.seed(1)

for n_feature in n_features:
    print("----- sklearn data with {} features -----".format(n_feature))
    if n_feature > 1:
        n_clusters = 2
    else:
        n_clusters = 1

    x, y = make_classification(1000000, n_informative=n_feature, n_features=n_feature, n_classes=2, n_clusters_per_class=n_clusters ,n_redundant=0, flip_y=0.0, class_sep=1.0)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.99)
    x_test1, x_test2, y_test1, y_test2 = train_test_split(x_test, y_test, test_size=0.5)
    
    x_train, y_train = torch.from_numpy(x_train).float(), torch.from_numpy(y_train).float()
    x_test1, y_test1 = torch.from_numpy(x_test1).float(), torch.from_numpy(y_test1).float()
    x_test2, y_test2 = torch.from_numpy(x_test2).float(), torch.from_numpy(y_test2).float()
    
    x_train.requires_grad = True
    x_test1.requires_grad = True
    
    w, b = train(x_train, y_train)

    y_pred = sigmoid(torch.matmul(x_test2, w) + b)
    y_pred[y_pred >= 0.5] = 1
    y_pred[y_pred < 0.5] = 0
    tn, fp, fn, tp = eval_model(y_test2, y_pred.detach())
    
    print("Original TPR: {}".format(tp))
    print("Original FPR: {}".format(fp))
    print("Original TNR: {}".format(tn))
    print("Original FNR: {}".format(fn))
    
    new_w, new_b = update_model_online(w, b, 2000)
    
    y_pred = sigmoid(torch.matmul(x_test2, new_w) + new_b)
    y_pred[y_pred >= 0.5] = 1
    y_pred[y_pred < 0.5] = 0
    tn, fp, fn, tp = eval_model(y_test2, y_pred.detach())
    
    print("Updated TPR: {}".format(tp))
    print("Updated FPR: {}".format(fp))
    print("Updated TNR: {}".format(tn))
    print("Updated FNR: {}".format(fn))

----- sklearn data with 1 features -----
tensor(1.3298, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0403, grad_fn=<BinaryCrossEntropyBackward>)
Original TPR: 0.5004181818181819
Original FPR: 0.0011818181818181819
Original TNR: 0.4983717171717172
Original FNR: 2.8282828282828282e-05
True
Updated TPR: 0.5004121212121212
Updated FPR: 0.0010686868686868686
Updated TNR: 0.4984848484848485
Updated FNR: 3.434343434343434e-05
----- sklearn data with 2 features -----
tensor(1.1548, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.2934, grad_fn=<BinaryCrossEntropyBackward>)
Original TPR: 0.4347252525252525
Original FPR: 0.044676767676767676
Original TNR: 0.45546868686868686
Original FNR: 0.06512929292929293
True
Updated TPR: 0.493220202020202
Updated FPR: 0.31825454545454546
Updated TNR: 0.1818909090909091
Updated FNR: 0.0066343434343434345
----- sklearn data with 3 features -----
tensor(1.0458, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5982, grad_fn=<BinaryCrossEntropyBackward>)
Original 